In [1]:
import sqlite3  # sqlite3 모듈 임포트
import pandas as pd  # pandas 모듈 임포트

## Create Database

In [ ]:
# 엑셀 파일 경로
excel_file = 'sqltut.xlsx'
# 데이터베이스 파일 경로
database_file = 'df3.db'
# 엑셀 파일을 판다스 데이터프레임으로 읽기 (모든 시트)
xls = pd.ExcelFile(excel_file)
# SQLite 데이터베이스 연결 생성
conn = sqlite3.connect(database_file)
# 각 시트를 반복하면서 데이터베이스에 테이블로 저장
for sheet_name in xls.sheet_names:
    # 각 시트를 데이터프레임으로 변환
    df = pd.read_excel(xls, sheet_name=sheet_name)
    # 각 컬럼별로 map 함수를 사용하여 변환
    for col in df.columns:
        if df[col].dtype == 'object':  # 문자열 데이터 타입 확인
            df[col] = df[col].map(lambda x: x.replace('\xa0', '').replace('\xa9', '') if isinstance(x, str) else x)
    # 데이터프레임을 SQLite 데이터베이스 테이블로 저장
    df.to_sql(sheet_name, conn, if_exists='replace', index=False)

In [3]:
db_url = 'sqlite:///df3.db'

conn = sqlite3.connect(database_file)
cursor = conn.cursor()

## SELECT

In [4]:
def query(cursor, query: str):
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

In [ ]:
query(cursor, "SELECT * FROM Customers")

In [ ]:
query(cursor, "SELECT CustomerName, City FROM Customers")

In [ ]:
query(cursor, "SELECT DISTINCT Country FROM Customers")

In [87]:
query(cursor, "SELECT COUNT(DISTINCT Country) FROM Customers")

(20,)


In [ ]:
# Numeric field에서는 single quotes를 사용하지 않아도 됨.
# Text field에서는 double quotes와 혼동하지 않도록 single quotes를 사용해야 함.

query(cursor, "SELECT * FROM Customers WHERE Country='Mexico'")

In [ ]:
query(cursor, "SELECT * FROM Customers WHERE CustomerID > 80;")

In [ ]:
# ORDER BY 쿼리 실행
query(cursor, "SELECT * FROM Products ORDER BY Price;")

In [ ]:
# Numeric Type에서는 숫자의 오름/내림차순으로 정렬 가능
# Text Type에서는 알파벳의 사전순/역순으로 정렬 가능
# ORDER BY는 중첩해서 사용할 수 있음.

query(cursor, "SELECT * FROM Products ORDER BY Price DESC;")

In [ ]:
query(cursor, "SELECT * FROM Customers ORDER BY Country ASC, CustomerName DESC;")

In [93]:
query(cursor, "SELECT * FROM Customers WHERE Country='Spain' AND CustomerName Like 'G%';")

('29', 'Galería del gastrónomo', 'Eduardo Saavedra', 'Rambla de Cataluña, 23', 'Barcelona', '08022', 'Spain')
('30', 'Godos Cocina Típica', 'José Pedro Freyre', 'C/ Romero, 33', 'Sevilla', '41101', 'Spain')


In [94]:
query(cursor, "SELECT * FROM Customers WHERE Country='Germany' AND City='Berlin' AND PostalCode > 12000;")

('1', 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')


In [95]:
query(
    cursor, 
    """
    SELECT * 
    FROM Customers
    WHERE Country='Spain' AND (CustomerName LIKE 'G%' or CustomerName LIKE 'R%');      
    """
)

('29', 'Galería del gastrónomo', 'Eduardo Saavedra', 'Rambla de Cataluña, 23', 'Barcelona', '08022', 'Spain')
('30', 'Godos Cocina Típica', 'José Pedro Freyre', 'C/ Romero, 33', 'Sevilla', '41101', 'Spain')
('69', 'Romero y tomillo', 'Alejandra Camino', 'Gran Vía, 1', 'Madrid', '28001', 'Spain')


In [ ]:
# 괄호를 작성하지 않은 경우에는 AND -> OR 순으로 쿼리가 진행됨
# Spain이면서 G로 시작하는 사람이거나, R로 시작하는 사람을 리턴하는 쿼리

query(
    cursor, 
    """
    SELECT * 
    FROM Customers
    WHERE Country='Spain' AND CustomerName LIKE 'G%' or CustomerName LIKE 'R%'; 
    """
)

In [ ]:
query(
    cursor, 
    """
    SELECT * 
    FROM Customers 
    WHERE Country IN ('Germany', 'Spain');
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE NOT Country='Spain';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE CustomerID NOT BETWEEN 10 AND 60;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE City NOT IN ('Paris', 'London');
    """
)

## INSERT

In [ ]:
# 기존 테이블 데이터 백업
cursor.execute("SELECT * FROM Customers")
rows = cursor.fetchall()

# 기존 테이블 구조 확인
cursor.execute("PRAGMA table_info(Customers);")
columns = cursor.fetchall()
print("Existing Customers table columns:")
for column in columns:
    print(column)

print("\nExisting Customers table data:")
for row in rows:
    print(row)

In [123]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers_new (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerName TEXT NOT NULL,
        ContactName TEXT,
        Address TEXT,
        City TEXT,
        PostalCode TEXT,
        Country TEXT
    )
""")
conn.commit()

In [124]:
for row in rows:
    cursor.execute("""
        INSERT INTO Customers_new (CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (int(row[0]), row[1], row[2], row[3], row[4], row[5], row[6])
    )
conn.commit()

In [125]:
cursor.execute("DROP TABLE Customers")
conn.commit()

In [126]:
cursor.execute("ALTER TABLE Customers_new RENAME TO Customers")
conn.commit()

In [127]:
cursor.execute("""
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
""")

query(cursor, "SELECT * FROM Customers WHERE CustomerName='Cardinal';")

(92, 'Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway')


In [130]:
cursor.execute("""
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES
    ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
    ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');            
""")

query(cursor, "SELECT * FROM Customers ORDER BY CustomerID DESC LIMIT 3;")

(96, 'Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK')
(95, 'Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway')
(94, 'Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK')


In [133]:
# NULL값은 말그대로 데이터가 존재하지 않는 상태를 의미하며, 비교/동등 연산자를 통해 쿼리할 수 없음.
# IS NULL, IS NOT NULL을 통해 NULL값을 쿼리할 수 있음.

query(
    cursor,
    """
    SELECT CustomerName, ContactName, Address 
    FROM Customers
    WHERE Address IS NULL;
    """
)


## Update

In [5]:
query(
    cursor,
    """
    UPDATE Customers
    SET ContactName='Alfred Schmidt', City='Frankfurt'
    WHERE CustomerID=1;
    """
)

In [6]:
# Update에서 Where 절을 작성하지 않으면 모든 행이 업데이트 된다는 것을 주의해야 함.

query(
    cursor,
    """
    UPDATE Customers
    SET ContactName='Juan'
    WHERE Country='Mexico';
    """
)

## Delete

In [7]:
# Delete에서 Where 절을 작성하지 않으면 모든 행이 삭제된다는 것을 주의해야 함.
# Drop Table은 테이블 자체를 삭제하는 것이므로 주의해야 함. (둘이 다름)

query(
    cursor,
    """
    DELETE FROM Customers
    Where CustomerName='Alfreds Futterkiste';
    """
)

In [14]:
query(
    cursor,
    """
    SELECT * FROM Customers LIMIT 3;
    """
)

print()
query(
    cursor,
    """
    SELECT * FROM Customers 
    FETCH FIRST 3 ROWS ONLY;
    """
)

(2, 'Ana Trujillo Emparedados y helados', 'Juan', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
(3, 'Antonio Moreno Taquería', 'Juan', 'Mataderos 2312', 'México D.F.', '05023', 'Mexico')
(4, 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
